### Cab-Driver Agent

In [1]:
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle
import pylab
import os
import tensorflow

# for building DQN model
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

# for plotting graphs
import matplotlib.pyplot as plt

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\sdasgupt\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\sdasgupt\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\sdasgupt\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\sdasgupt\AppData\Local\Continuum\anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\sdasgupt\AppData\Local\Continuum\anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The

TypeError: can only concatenate str (not "list") to str

In [ ]:
# Import the environment
from Env import CabDriver

#### Defining Time Matrix

In [ ]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")

In [ ]:
Time_matrix

In [ ]:
# create an environment object 
env = CabDriver()

#### Tracking the state-action pairs for checking convergence


In [ ]:
# Creating directories for saving the model and graph if the directories doesn't exist
dir_model = 'save_model'
if not os.path.exists(dir_model):
    os.makedirs(dir_model)

In [ ]:
def Q_state(state):
    str_state = ""
    for s in state:
        if s < 10:
            str_state = str_state + "0" + str(s)
        else:
            str_state = str_state + str(s)
    return str_state

In [ ]:
# Function to save q-value for tracking states-action pair
def save_tracking_states():
    initialise_tracking_states()

    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action] += [q[1] for q in Q_dict[state][action]]

In [ ]:
# Defining a function which will add new Q-values to the Q-dictionary. 
def add_to_dict(state, q_val, action, epi):
    state1 = Q_state(state)
    
    if action not in Q_dict[state1]:
        Q_dict[state1][action]= []    
    Q_dict[state1][action].append((epi, q_val))

In [ ]:
# Function to save q-value for tracking states-action pair
def initialise_tracking_states():
    sample_q_values = [('021101', (2, 4)), 
                       ('030905', (3, 5)), 
                       ('040502', (4, 3)), 
                       ('011604', (1, 3))]    #select any 4 Q-values
    for q_values in sample_q_values:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = []    #this is an array which will have appended values of that state-action pair for every 2000th episode         

In [ ]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# Initialise Q_dictionary and States_tracked (for convergence)
Q_dict = collections.defaultdict(dict)
States_track = collections.defaultdict(dict)

initialise_tracking_states()

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [ ]:
#Defining hyperparameters
LEN_EPISOD_DAY = 30          #length of an Episode               
LR = 0.001                   #learning rate
GAMMA = 0.91
epsilon=1
epsilon_min = 0.0001
epsilon_decay = 0.005
threshold = 600            #every these many episodes, the 4 Q-values will be stored/appended (convergence graphs)
policy_threshold = 700    #every these many episodes,the policy will be saved

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        # Define size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # Write here: Specify you hyper parameters for the DQN
        self.discount_factor = GAMMA
        self.learning_rate =  LR
        self.epsilon_min = epsilon_min
        self.epsilon_max = epsilon
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        
        self.batch_size = 32        
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # create main model and target model
        self.model = self.build_model()

    # approximate Q function using Neural Network
    def build_model(self):
        model = Sequential()
        # Write your code here: Add layers to your neural nets 
        # Hidden layers: 1, 2, 3, 4 
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
        #Output Layer
        model.add(Dense(self.action_size, activation='relu', kernel_initializer='he_uniform'))
        
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        model.summary
        return model


    def get_action(self, state,list_actions):
    # Write your code here:
    # get action from model using epsilon-greedy policy
    # Decay in ε after we generate each sample from the environment       
        q_value = self.model.predict(state)
        #explore -> Populate a random action from all possible actions
        if np.random.rand() <= self.epsilon:        
            action = random.choice(list_actions)
        #Pick the action with the highest q(s, a)
        else:
            action_q_values = [q_value[0][index] for index in list_actions]
            max_q_value = np.max(action_q_values)
            action = list(q_value[0]).index(max_q_value)

        return action, q_value[0][action]  
    
        
    def get_model_weights(self):
        return self.model.get_weights()  
        


    def append_sample(self, state, action, reward, next_state,terminal):
    # Write your code here:
    # save sample <s,a,r,s'> to the replay memory
        self.memory.append((state, action, reward, next_state, terminal))
        # Decay in ε after we generate each sample from the environment
        if self.epsilon_max > self.epsilon_min:
            self.epsilon_max *= self.epsilon_decay    

    
    # pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self):
        
        if len(self.memory) > self.batch_size:
            # Sample batch from the memory
            mini_batch = random.sample(self.memory, self.batch_size)
            batch_size = min(self.batch_size, len(self.memory))
            update_output = np.zeros((self.batch_size, self.state_size)) 
            update_input = np.zeros((self.batch_size, self.state_size))
            
            actions, rewards,terminal_state = [], [], []
            
            for i in range(self.batch_size):
                # Write your code from here
                state, action, reward, next_state,terminal = mini_batch[i]
                update_input[i] = state
                actions.append(action)
                rewards.append(reward)
                update_output[i] = next_state
                terminal_state.append(terminal)

            # 1. Predict the target from earlier model
                target = self.model.predict(update_input)

                # 2. Get the target for the Q-network
                target_qval = self.model.predict(update_output)  

            for i in range(self.batch_size):
                if terminal_state[i]:
                    target[i][actions[i]] = rewards[i]
                else: # non-terminal state
                    target[i][actions[i]] = rewards[i] + self.discount_factor * np.max(target_qval[i])

            # 4. Fit your model and track the loss values
            self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)

    def save(self, name):
        self.model.save(name)
        
    def save_model_graph(self, suffix):
        # convert model to JSON
        model_json = self.model.to_json()
        with open("./" + dir_model + "/cabdriver_model_" + suffix + ".json", "w") as json_file:
            json_file.write(model_json)

In [ ]:
Episodes = 5000

### DQN block

In [ ]:
#Initialize variables
scores, episodes = [], []
state = env.state_init
state_encode = env.state_encod_arch1(state)
state_size = len(state_encode)
action_size = len(env.action_space)
agent = DQNAgent(state_size, action_size)


for episode in range(Episodes):

    # Write code here
    # Call the environment
    # Call all the initialised variables of the environment
    terminal_state = False
    score = 0
    reward = 0
    total_days = 1
    prev_day = 0
    curr_day = 0
    env.reset()
    state = env.state_init    

    #Call the DQN agent
    
    
    while not terminal_state:
        
        # Write your code here
        
        # 1. Pick epsilon-greedy action from possible actions for the current state
        state_encode = env.state_encod_arch1(state)
        state_encode = np.reshape(state_encode, [1, state_size])
        
        possib_actions, action_list = env.requests(state)
        action, q_value = agent.get_action(state_encode, possib_actions)
        
        # 2. Evaluate your reward and next state
        reward = env.reward_func(state, env.action_space[action], Time_matrix)
        next_state = env.next_state_func(state, env.action_space[action], Time_matrix)
        next_state_enc = env.state_encod_arch1(next_state)
        next_state_enc = np.reshape(next_state_enc, [1, state_size]) 
        
        # 3. Append the experience to the memory
        agent.append_sample(state_encode, action, reward, next_state_enc, terminal_state)
        
        add_to_dict(state, q_value, env.action_space[action], episode)
        
        # 4. Train the model by calling function agent.train_model
        agent.train_model()
        
        # 5. Keep a track of rewards, Q-values, loss
        score += reward
        prev_day = state[2]
        state = next_state
        curr_day = state[2]
        if prev_day != curr_day:
            total_days = total_days + 1
            
        if total_days > LEN_EPISOD_DAY:
            terminal_state = True
            
        if terminal_state == True:
            scores.append(score)
            episodes.append(episode)  
    
    # epsilon decay
#    agent.epsilon =  epsilon_decay(Episodes, episode)
        # epsilon decay
        agent.epsilon = abs(np.exp((-np.pi/Episodes)*episode)) 
        
    print("Episode:", episode, "  score:", score, "  memory length:", len(agent.memory), "  epsilon:", agent.epsilon)

    #TRACKING Q-VALUES
    if ((episode+1) % threshold) == 0:   #every threshold episode
        save_tracking_states()
        save_obj(States_track,'States_tracked')

    
    # plot episode versus score 
    if episode % threshold == 0: 
        pylab.plot(episodes, scores)
        pylab.savefig("./" + dir_model + "/cardriver_dqn_" + str(score) + ".png")    
    
    #save the model 
    if episode % policy_threshold == 0:         
        # Model binary h5 file
        agent.save("./" + dir_model + "/cardriver_dqn_" + str(score) + ".h5")
        
        # Model pickle file
        save_obj(agent.get_model_weights(), "./" + dir_model + "/cardriver_dqn_" + str(score))
    
    #SAVING POLICY
    if ((episode+1)% policy_threshold ) == 0:  #every policy_threshold episodes, the Q-dict will be saved
        save_obj(Q_dict,'Policy')        
        

### Tracking Convergence

In [ ]:
# Q-Values for state-action pairs is obtained for diffrent episodes
#------------------------------------------------------------------
# Location C for 09:00 hours on FRIDAY
# result dictionary contains: 
# {(Action-Pair): [(Episode1, Q(s,a)), (Episode2, Q(s,a)),....]} 

Q_dict["030905"]

In [ ]:
# Q Value Tracking Location C for 09:00 hours on FRIDAY for action location C->D
# result dictionary contains: 
# [(Episode_ID: Q(s,a)), ...]} 

print("All actions taken from state 030905 : {}\n".format(Q_dict["031701"].keys()))

print("All Q-values for action (3, 5) :  (EpisodeID, Q-value)")
if (3, 5) in Q_dict["030905"]:
    print(Q_dict["030905"][(3, 5)])

In [ ]:
# Convergence graphs: Total Q-value per episode

pylab.plot(episodes, scores)

In [ ]:
# Q value for state ["030905"] and action [(3, 5)]
Q_dict["030905"][(3, 5)]

In [ ]:
States_track

In [ ]:
# Convergence of Q-Value for state 03-10-06 and action (3, 5)
pylab.plot(States_track['030905'][(3, 5)])

#### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
time = np.arange(0,10000)
epsilon = []
for i in range(0,10000):
    epsilon.append(0 + (1 - 0) * np.exp(-0.0009*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()